In [ ]:
import os
os.environ['CURL_CA_BUNDLE'] = ""

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

llm = Ollama(model="llama3:8b-instruct-q5_K_M", request_timeout=120.0)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
response = llm.complete("do you like drake or kendrick better?")

print(response)

In [ ]:
# !mkdir data
# !wget "https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1" -O data/drake_kendrick_beef.pdf
# !wget "https://www.dropbox.com/scl/fi/nts3n64s6kymner2jppd6/drake.pdf?rlkey=hksirpqwzlzqoejn55zemk6ld&st=mohyfyh4&dl=1" -O data/drake.pdf
# !wget "https://www.dropbox.com/scl/fi/8ax2vnoebhmy44bes2n1d/kendrick.pdf?rlkey=fhxvn94t5amdqcv9vshifd3hj&st=dxdtytn6&dl=1" -O data/kendrick.pdf

In [ ]:
from llama_index.core import SimpleDirectoryReader

docs_kendrick = SimpleDirectoryReader(input_files=["data/kendrick.pdf"]).load_data()
docs_drake = SimpleDirectoryReader(input_files=["data/drake.pdf"]).load_data()
docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

In [ ]:
from llama_index.core import VectorStoreIndex

drake_index = VectorStoreIndex.from_documents(docs_drake)
drake_query_engine = drake_index.as_query_engine(similarity_top_k=3)

kendrick_index = VectorStoreIndex.from_documents(docs_kendrick)
kendrick_query_engine = kendrick_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

drake_tool = QueryEngineTool(
    drake_index.as_query_engine(),
    metadata=ToolMetadata(
        name="drake_search",
        description="Useful for searching over Drake's life.",
    ),
)

kendrick_tool = QueryEngineTool(
    kendrick_index.as_query_engine(),
    metadata=ToolMetadata(
        name="kendrick_summary",
        description="Useful for searching over Kendrick's life.",
    ),
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [drake_tool, kendrick_tool],
    llm=llm,  # llama3-70b
    verbose=True,
)


In [ ]:
[print(i) for i in query_engine.get_prompts().keys()]

In [ ]:
print(query_engine.get_prompts()['question_gen:question_gen_prompt'].template)

In [ ]:
print(query_engine.get_prompts()['response_synthesizer:text_qa_template'].get_template())


In [ ]:
print(query_engine.get_prompts()['response_synthesizer:refine_template'].get_template())


In [ ]:
try:
    response = query_engine.query("Which albums did Drake release in his career?")

    print(response)
    
except Exception as e:
    print("\nERROR - unable to process")

In [ ]:
del llm

In [ ]:
llm = Ollama(model='llama3:8b-instruct-q5_K_M', request_timeout=240.0)
    
Settings.llm = llm

query_engine = SubQuestionQueryEngine.from_defaults(
[drake_tool, kendrick_tool],
llm=llm,
verbose=True,
)

response = query_engine.query("Which albums did Drake release in his career?")

print(response)

In [ ]:
A = query_engine.get_prompts()['question_gen:question_gen_prompt']

print(A.template)

In [ ]:
llama_3_prompt_str = (
    '''<|begin_of_text|><|start_header_id|>system<|end_header_id|>Given a user question, and a list of tools, output a list of relevant sub-questions in 'json' markdown that when composed can help answer the full user question:
    OUTPUT RESULT IN JSON FORMAT AS SHOWN IN EXAMPLE.

# Example 1

[Tools]
```json
{{
    "uber_10k": "Provides information about Uber financials for year 2021",
    "lyft_10k": "Provides information about Lyft financials for year 2021"
}}
```

[User Question]
Compare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021


[Output]
```json
{{
    "items": [
        {{
            "sub_question": "What is the revenue growth of Uber",
            "tool_name": "uber_10k"
        }},
        {{
            "sub_question": "What is the EBITDA of Uber",
            "tool_name": "uber_10k"
        }},
        {{
            "sub_question": "What is the revenue growth of Lyft",
            "tool_name": "lyft_10k"
        }},
        {{
            "sub_question": "What is the EBITDA of Lyft",
            "tool_name": "lyft_10k"
        }}
    ]
}}
```

# Example 2
[Tools]
```json
{tools_str}
```

<|eot_id|><|start_header_id|>user<|end_header_id|>

[User Question]
{query_str}

[Output]<|eot_id|><|start_header_id|>assistant<|end_header_id|>'''
)

In [ ]:
query_engine.get_prompts()['question_gen:question_gen_prompt']

In [ ]:
A = query_engine.get_prompts()['question_gen:question_gen_prompt']
A.template = llama_3_prompt_str
query_engine.update_prompts(
    {"question_gen:question_gen_prompt": A})

In [ ]:
# A = query_engine.get_prompts()['question_gen:question_gen_prompt']

# print(A.template)

In [ ]:
llm = Ollama(model='llama3:8b-instruct-q5_K_M', request_timeout=240.0)
    
Settings.llm = llm

query_engine = SubQuestionQueryEngine.from_defaults(
[drake_tool, kendrick_tool],
llm=llm,
verbose=True,
)

A = query_engine.get_prompts()['question_gen:question_gen_prompt']
A.template = llama_3_prompt_str
query_engine.update_prompts(
    {"question_gen:question_gen_prompt": A})

response = query_engine.query("Which albums did Drake release in his career?")

print(response)

In [ ]:
# query_engine.get_prompts()

In [ ]:
models = ['llama3:8b-instruct-q5_K_M', 'llama3:8b-instruct-q8_0']
# models = ['llama3:70b-instruct-q2_K']
models = ['llama3:8b-instruct-q8_0']
prompts = [llama_3_prompt_str]
prompt_name = ['llama_3_prompt_str']


for model in models:
    
    del llm
    
    llm = Ollama(model=model, request_timeout=120.0)
    
    Settings.llm = llm

    query_engine = SubQuestionQueryEngine.from_defaults(
    [drake_tool, kendrick_tool],
    llm=llm,
    verbose=True,
    )
    
    
#     print('\n\n---------------')
#     print('Model:'+model)
#     print('Prompt:default')
    
#     try:
#         response = query_engine.query("Which albums did Drake release in his career?")
#         print(str(response))
    
#     except Exception as e:
#         print("\nERROR - unable to process")
        
        
    for i, prompt in enumerate(prompts):
        
        print('\n\n')
        print('Model:'+model)
        print('Prompt:'+prompt_name[i])

        query_engine = SubQuestionQueryEngine.from_defaults(
        [drake_tool, kendrick_tool],
        llm=llm,
        verbose=True,
        )

        A = query_engine.get_prompts()['question_gen:question_gen_prompt']
        A.template = prompt
        query_engine.update_prompts(
            {"question_gen:question_gen_prompt": A})

        try:
            response = query_engine.query("Which albums did Drake release in his career?")
            print(str(response))

        except Exception as e:
            print("\nERROR - unable to process")
#             print(e)
        